<h1>Dates table</h1>

<h3>IMPORTS</h3>

In [1]:
#IMPORTS
import pandas as pd
import os
from google.cloud.sql.connector import Connector, IPTypes
import pymysql
import sqlalchemy
from zipfile import ZipFile
from datetime import datetime, timedelta

<h3>METHODS</h3>

In [2]:
def get_dates_until_current(start_date_str):
    """
    Generates a list of dates from a given start date up to the current date.

    Args:
        start_date_str (str): The start date in 'YYYY-MM-DD' format.

    Returns:
        list: A list of datetime.date objects representing the dates in the range.
    """
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d').date()
    current_date = datetime.now().date()
    date_list = []

    # Iterate from the start_date until it exceeds the current_date
    while start_date <= current_date:
        date_list.append(start_date)
        start_date += timedelta(days=1) # Increment by one day

    return date_list



In [3]:
def create_dates_df(dates_in_range):

    # Create DataFrame
    dates_df = pd.DataFrame(dates_in_range, columns=["Date"])

    #convert to pandas datetime format
    dates_df["Date"] = pd.to_datetime(dates_df["Date"])

    #extract granularity from dates
    dates_df["Month"] = dates_df["Date"].dt.month
    dates_df["Year"] = dates_df["Date"].dt.year
    dates_df["Quarter"] = dates_df["Date"].dt.quarter

    return dates_df

<h3>Establish file paths & connections</h3>

In [4]:
#Connection to database
def connect_with_connector() -> sqlalchemy.engine.base.Engine:
    """
    Initializes a connection pool for a Cloud SQL instance of MySQL.

    Uses the Cloud SQL Python Connector package.
    """

    instance_connection_name = "esoteric-life-470902-k9:australia-southeast2:jyjam-weatherdata-db"
    db_user = "admin" 
    db_pass = "Password1!"
    db_name ="weather_data_database"

    ip_type = IPTypes.PRIVATE if os.environ.get("PRIVATE_IP") else IPTypes.PUBLIC

    # initialize Cloud SQL Python Connector object
    connector = Connector(ip_type=ip_type, refresh_strategy="LAZY")

    def getconn() -> pymysql.connections.Connection:
        conn: pymysql.connections.Connection = connector.connect(
            instance_connection_name,
            "pymysql",
            user=db_user,
            password=db_pass,
            db=db_name,
        )
        return conn

    pool = sqlalchemy.create_engine(
        "mysql+pymysql://",
        creator=getconn,
    )
    return pool


In [5]:
#connection to database
db_conn = connect_with_connector()
db_table = 'Dates'

<h3>CREATE DATAFRAME</h3>

In [6]:
#Get all dates from start of record to current date
start_date_input = '2015-01-01' #YYYY-MM-DD format

#check if earliest date
sql_earliest_date = sqlalchemy.text("SELECT Min(Date) AS OldestDate from weather_data;")

with db_conn.connect() as cursor:
    try:
        # Execute the DELETE statement
        result = cursor.execute(sql_earliest_date)

        print(f"QUERY complete: {sql_earliest_date}")

    except Exception as e:
        print(f"Error -> {e}")

    finally:
        # Close the cursor
        cursor.close()


rows = result.fetchall()

for row in rows:
    print(row)


QUERY complete: SELECT Min(Date) AS OldestDate from weather_data;
(None,)


<h3>INSERT ALL Data into weather_data table</h3>

In [8]:
dates_in_range = get_dates_until_current(start_date_input)

dates_df = create_dates_df(dates_in_range)

with db_conn.connect() as cursor:
    dates_df.to_sql(db_table, cursor, if_exists='append', index=False)

<h3>RESET TABLE ROWS</h3>

In [ ]:
# cursor = db_conn.cursor()


# sql_create_unique_table = "CREATE TABLE dates_temp AS SELECT DISTINCT * FROM Dates"
# sql_drop_old_table = "DROP TABLE Dates"
# sql_create_new_table = "CREATE TABLE Dates AS SELECT DISTINCT * FROM dates_temp"
# sql_drop_temp_table = "DROP TABLE dates_temp"

# try:
#     # Execute the DELETE statement
#     cursor.execute(sql_create_unique_table)
#     cursor.execute(sql_drop_old_table)
#     cursor.execute(sql_create_new_table)
#     cursor.execute(sql_drop_temp_table)


#     # Commit the changes to the database
#     db_conn.commit()

#     print(f"Distinct '{db_table}' table has been created.")

# except sqlite3.Error as e:
#     print(f"Error creating distinct table: {e}")

# finally:
#     # Close the cursor
#     cursor.close()

AttributeError: 'Engine' object has no attribute 'cursor'